# Use IBM watsonx and `LangChain`

## Notebook content

This notebook contains the steps and code to demonstrate simple sequential chain using langchain integration with IBM watsonx models.

Some familiarity with Python is helpful. This notebook uses Python 3.10.

### Contents

1. [Setup](#setup_environment)
1. [Initialize the model](#initialize_model)
1. [watsonxLLM interface](#watsonxllm)
1. [Simple sequential chain experiment](#experiment)

<a id="setup_environment"></a>
## 1. Set up the environment

In [ ]:
!pip install -U ibm-watson-machine-learning --quiet
!pip install langchain --quiet

In [ ]:
credentials = {
    "url": "URL",
    "apikey": "API_KEY"
}

In [ ]:
project_id = 'PROJECT_ID'

### 1.2 List available models

All avaliable models are presented under `ModelTypes` class.

In [ ]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.value for model in ModelTypes])

In [ ]:
model_id_llama = ModelTypes.LLAMA_2_70B_CHAT
model_id_flan_t5 = ModelTypes.FLAN_T5_XXL

### 1.3 Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to documentation under `GenTextParamsMetaNames` class.

In [ ]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}

<a id="initialize_model"></a>
## 2. Initialize the model

Initialize the `Model` class with previous set params.

In [ ]:
from ibm_watson_machine_learning.foundation_models import Model

llama_model = Model(
    model_id=model_id_llama, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)

flan_t5_model = Model(
    model_id=model_id_flan_t5,
    credentials=credentials,
    project_id=project_id)

<a id="watsonxllm"></a>
## 3. watsonxLLM interface

`WatsonxLLM` is a wrapper around watsonx.ai models that provide chain integration around the models.

**Action:** For more details about `CustomLLM` check the [LangChain documentation](https://python.langchain.com/docs/modules/model_io/models/llms/custom_llm)

#### Initialize the `WatsonxLLM` class.

In [ ]:
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

llama_llm = WatsonxLLM(model=llama_model)
flan_t5_llm = WatsonxLLM(model=flan_t5_model)

You can print all set data about the WatsonxLLM object using the `dict()` method.

In [ ]:
llama_llm.dict()

<a id="experiment"></a>
## 4. Sequential chain experiment
The simplest type of sequential chain is called a `SimpleSequentialChain`, in which each step has a single input and output and the output of one step serves as the input for the following step.

The experiment will consist in generating a random question about any topic and answer the following question.

An object called `PromptTemplate` assists in generating prompts using a combination of user input, additional non-static data, and a fixed template string.

In our case we would like to create two `PromptTemplate` objects which will be responsible for creating a random question and answering it.

### Example 1:

In [ ]:
from langchain import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["topic"], 
    template="Generate a random question about {topic}: Question: "
)
prompt_2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

In [ ]:
from langchain.chains import LLMChain

prompt_to_llama = LLMChain(llm=llama_llm, prompt=prompt_1)

In [ ]:
llama_to_t5 = LLMChain(llm=flan_t5_llm, prompt=prompt_2)

This is the overall chain where we run `prompt_to_llama` and `llama_to_t5` chains in sequence.

In [ ]:
from langchain.chains import SimpleSequentialChain

qa = SimpleSequentialChain(chains=[prompt_to_llama, llama_to_t5], verbose=True)

In [ ]:
qa.run('brazil')

### Example 2:

In [ ]:
prompt_2_1 = PromptTemplate(
    input_variables=["customer_complaint"], 
    template="From the following customer complaint, extract 3 factors that caused the customer to be unhappy. Put each factor on a new line. Customer complaint: {customer_complaint}. Numbered list of complaints: "
)

prompt_2_2 = PromptTemplate(
    input_variables=["list_of_complaints"], 
    template="Does the following statements contain the concept of identify theft?: {list_of_complaints}"
)

In [ ]:
prompt_to_llama_2 = LLMChain(llm=llama_llm, prompt=prompt_1)

llama_to_t5_2 = LLMChain(llm=flan_t5_llm, prompt=prompt_2)

In [ ]:
qa_2 = SimpleSequentialChain(chains=[prompt_to_llama_2, llama_to_t5_2], verbose=True)

In [ ]:
customer_complaint = "I am writing you this statement to delete the following information on my credit report. The items I need deleted are listed in the report. I am a victim of identity thief, I demand that you remove these errors to correct my report immediately! I have reported this to the federal trade commission and have attached the federal trade commission affidavit. Now that I have given you the following information, please correct my credit report or I shall proceed with involving my attorney!"

In [ ]:
qa_2.run(customer_complaint)